In [1]:
!pip install numpy
!pip install pandas
!pip install tqdm
!pip install scikit-learn
!pip install catboost

^C





You should consider upgrading via the 'D:\conda\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'D:\conda\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'D:\conda\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'D:\conda\python.exe -m pip install --upgrade pip' command.


In [ ]:
!pip install autokeras

In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_squared_error
from catboost import CatBoostRegressor

pd.set_option('display.max_columns', 200)

In [2]:
X_train_dataset = pd.read_csv("X_train.csv")
y_train_dataset = pd.read_csv("y_train.csv")

In [3]:
df = pd.merge(X_train_dataset, y_train_dataset, on=["engine_id", "flight_datetime", "flight_phase"])

In [4]:
import autokeras as ak

In [ ]:
parameters=pd.read_csv("parameter_list.csv")

In [ ]:
parameters

In [ ]:
#найти параметры для которых  engine_aircraft равно AIRCRAFT
new_par=parameters.loc[parameters['engine_aircraft']=='AIRCRAFT']
nes_par=new_par['parameter_name']

In [9]:
def get_score_by_parameter2(predict_field):
    df = pd.merge(X_train_dataset, y_train_dataset, on=["engine_id", "flight_datetime", "flight_phase"])
    df = df[df[predict_field].notna()]
    df = df.fillna(-100)
    X = df.drop(columns=[predict_field])
    X = X.drop(columns=[
        "engine_id", "aircraft_id", "flight_datetime",
        "flight_phase", "engine_family", "engine_type", "manufacturer",
        "aircraft_family", "aircraft_type", "aircraft_grp", "ac_manufacturer"])
    y = df[predict_field]
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
    
    # Initialize the structured data regressor.
    reg = ak.StructuredDataRegressor(
        overwrite=True, max_trials=3
        )  # It tries 3 different models.
    reg.fit(X_train, y_train, epochs=10)
    struct_predicted = reg.predict(X_test)
    struct_score = mean_squared_error(y_test, struct_predicted)
    struct_predicted = reg.predict(X)
    return struct_score

In [ ]:
def get_score_by_parameter(predict_field):
    df = pd.merge(X_train_dataset, y_train_dataset, on=["engine_id", "flight_datetime", "flight_phase"])
    df = df[df[predict_field].notna()]
    df = df.fillna(-100)
    X = df.drop(columns=[predict_field])
    X = X.drop(columns=[
        "engine_id", "aircraft_id", "flight_datetime",
        "flight_phase", "engine_family", "engine_type", "manufacturer",
        "aircraft_family", "aircraft_type", "aircraft_grp", "ac_manufacturer"])
    y = df[predict_field]
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
    
    model = CatBoostRegressor(iterations=100, verbose=False)
    #model = LogisticRegression(n_jobs=-1, penalty='none')
    model.fit(X_train, y_train, epochs=10)
    catboost_predicted = model.predict(X_test)
    catboost_score = mean_squared_error(y_test, catboost_predicted)
    catboost_predicted = model.predict(X)
    return catboost_score

In [ ]:
scores = pd.DataFrame(columns=["parameter", "rmse", ".0", ".1", ".5", ".9", "1"])
output_parameters = y_train_dataset.columns.drop(['flight_datetime', 'flight_phase', 'engine_id'])

for param in tqdm(output_parameters):
    try:
        score = get_score_by_parameter2(param)
        qua = df[param].quantile([.0, .1, .5, .9, 1])
        qua = list(qua)
        scores.loc[len(scores)] = [param, score] + qua
    except:
        continue

In [ ]:
output_parameters

In [ ]:
scores

In [ ]:
nesessary_parameters=['flight_datetime',
                      'flight_phase',
                      'engine_id',
                      'BRAT',
                      'DEGT',
                      'DELFN',
                      'DELN1',
                      'DELVSV',
                      'DPOIL',
                      'EGTC',
                      'EGTHDM',
                      'EGTHDM_D',
                      'GEGTMC',
                      'GN2MC',
                      'GPCN25',
                      'GWFM',
                      'PCN12',
                      'PCN12I',
                      'PCN1AR',
                      'PCN1BR',
                      'PCN1K','PCN2C','SLOATL','SLOATL_D','VSVNOM','WBE','WBI','WFMP',
                      'ZPCN25_D','ZT49_D','ZTLA_D','ZTNAC_D','ZWF36_D']



In [ ]:
np.shape(nesessary_parameters)